# <u> Automatic Differentiation in PyTorch </u>

In this notebook, we will:
1. Review basic PyTorch tensors
2. Introduce automatic differentiation (`autograd`)
3. Revisit our toy example from the slides
4. Use autodiff for optimization

The goal is **intuition**, not speed or deep learning tricks.

## PyTorch tensors (the basics)

A PyTorch `Tensor` is similar to a NumPy array:
- N-dimensional
- Supports vectorized operations
- Can live on CPU or GPU (if CUDA is available)

Unlike NumPy arrays, PyTorch tensors can also:
- **Track operations (Needed for AD)**
- **Compute derivatives automatically**

In [ ]:
import torch # import the PyTorch library
import numpy as np # import the NumPy library

x = torch.tensor([1.0, 2.0, 3.0]) # create a 1D tensor 
print(x)
print(type(x))

In [ ]:

y = np.array([4.0, 5.0, 6.0]) # create a NumPy array
print(y)
print(type(y))

x = torch.from_numpy(y) # convert NumPy array to PyTorch tensor
print(x)
print(type(x))

y = x.numpy() # convert PyTorch tensor back to NumPy array
print(y)
print(type(y))

PyTorch treats **scalars** as 0-dimensional tensors and **vectors** as 1-D tensors.

This matters for autodiff:
- Gradients are defined for **scalars**
- Most ML losses are scalars

In [ ]:
x = torch.tensor(2.0)
print(x)
print(x.shape)

### Tracking derivatives with `requires_grad`

To tell PyTorch that we want derivatives with respect to a tensor,
we set: `requires_grad=True` when creating it.

From this point on, PyTorch:
- Builds a **computational graph**
- Stores how each operation depends on this tensor

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
print(x)

### Toy example

From the slides, we consider the function:

y = sin(x) * x^2

We will:
1. Evaluate this function
2. Compute its derivative using autodiff
3. Compare against the analytic result

In [ ]:
# our function y = sin(x) * x^2
# Notice the use of torch sine function
def f(x): 
    return torch.sin(x) * x**2

### Forward pass

First, we simply *evaluate* the function.

This is just normal numerical computation.

In [ ]:
x = torch.tensor(1.234, requires_grad=True)
y = f(x)
print(f"y = {y.item()}")

### Backward pass (reverse-mode autodiff)

Calling `.backward()`:
- Starts from the output `y`
- Walks backward through the computational graph
- Applies the chain rule automatically

This computes:
dy/dx and stores it in `x.grad`

In [ ]:
y.backward() # do the backward pass through f(x)
print("dy/dx =", x.grad.item()) # print the gradient, the .item() gets the raw value

### Consistency check: analytic derivative

Analytically:

df/dx = d[sin(x)x^2]/dx = 2x * sin(x) + x^2 * cos(x)

Autodiff should match this exactly (up to floating-point precision).

In [ ]:
x_val = x.detach().numpy() # must detach the PyTorch tensor before converting to a NumPy array if it is tracking gradients
analytic = 2*x_val * np.sin(x_val) + (x_val**2) * np.cos(x_val)

print(f"autodiff : {x.grad}")
print(f"analytic : {analytic}")
print(f"difference: {abs(x.grad.numpy() - analytic)}")

In [ ]:
x = torch.tensor(1.234, requires_grad=True) # redefine x and its gradient

f(x).backward() # first backward pass (computes dy/dx)
print("after first backward:", x.grad)

f(x).backward() # second backward pass (computes dy/dx again and adds to existing gradient)
print("after second backward:", x.grad)

### Clearing gradients

In optimization loops, we usually clear gradients explicitly using:
x.grad.zero_()

In [ ]:
x.grad.zero_() # clear existing gradient
f(x).backward() # compute gradient again
print("after zero_ then backward:", x.grad.item())

### Optimization using autodiff

Now we use gradients to *optimize* a parameter.

This is exactly what training a neural network does —
just with many more parameters.

In [ ]:
y_target = torch.tensor(1.0) # the target value for y, i.e., we want to find x such that f(x) = 1.0

x = torch.tensor(2.5, requires_grad=True) # initial guess for x
lr = 1E-3 # learning rate, this is a hyperparameter that defines the step size in each iteration

# optimization loop, lets run for 100 steps
for step in range(100):

    # evaluate function and compute loss
    y = f(x) # compute current value of y
    loss = (y - y_target)**2 # mean squared error loss
    loss.backward() # compute gradient dy/dx

    # the optimizer is not part of the computational graph, so we use torch.no_grad() 
    # on the update step. This prevents PyTorch from tracking operations on x during the update.
    with torch.no_grad(): # disable gradient tracking for the update step
        x -= lr * x.grad # gradient descent update, move x in the direction of negative gradient
        x.grad.zero_() # clear gradients for the next iteration

    # print progress every 5 steps
    if step % 5 == 0 or step == 99:
        print(f"step {step:02d}  x={x.item(): .4f}  y={y.item(): .4f}  log(loss)={np.log10(loss.item()): .4f}")

### Takeaway

PyTorch Automatic differentiation:
- Tracks computation through backward pass
- Applies the chain rule automatically
- Scales to millions of parameters

This is the foundation of modern scientific ML and deep learning with PyTorch.

## `torch.nn` module
In practice, we usually define models as subclasses of `torch.nn.Module`. This container is just a Python object that knows how to compute a forward pass and holds trainable parameters (parameters that we want to fit).

In [ ]:
# import the neural network module from PyTorch
import torch.nn as nn

# define a simple model as a subclass of nn.Module
class ToyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.x = nn.Parameter(torch.tensor(2.5)) # define a trainable parameter x

    # the forward method defines the computation performed at every call
    def forward(self):
        return torch.sin(self.x) * self.x**2

In [ ]:
# initialize the model object
toymodel = ToyModel()

# perform a forward pass to compute the output
y = toymodel()
print(f"y = {y}")

print(f"model parameter x = {toymodel.x.item()}") # access the model parameter x, using item to get the Python number

# lets see the model parameters
for name, p in toymodel.named_parameters():
    print(f"parameter name: {name}, value: {p.item()}, requires_grad: {p.requires_grad}")

## `torch.optim` module
PyTorch provides an `optim` module that implements various optimization algorithms (e.g., SGD, Adam). This module simplifies the optimization process by handling parameter updates based on computed gradients.

In [ ]:
from torch import optim # import the optim module from PyTorch
y_target = torch.tensor(1.0) # target value for y, we want to find x such that model(x) = 1.0

# use the SGD optimizer from torch.optim instead of our manual gradient descent
toyoptimizer = torch.optim.SGD(toymodel.parameters(), lr=1E-3) # create an SGD optimizer to update model parameters

# optimization loop using the optimizer
for step in range(100):
    
    # forward pass and compute loss
    y = toymodel()
    loss = (y - y_target)**2

    # backward pass to compute gradients
    loss.backward()
    
    # update model parameters using the optimizer
    toyoptimizer.step()

    # clear existing gradients
    toyoptimizer.zero_grad()

    # print progress every 5 steps
    if step % 5 == 0 or step == 99:
        print(f"step {step:02d}  x={toymodel.x.item(): .4f}  y={y.item(): .4f}  log(loss)={np.log10(loss.item()): .4f}")


# <u> CPU vs GPU in PyTorch </u>

PyTorch tensors live on a **device**:
- CPU (default)
- GPU (CUDA), if available (Metal for Mac is in beta, not covered here but might be available and useful for Apple Silicon Macs)

Operations happen on the device where the tensors live.

In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

## Moving tensors between devices

We move tensors explicitly using `.to(device)`.

All tensors involved in a computation must live on the **same device**.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x = torch.tensor(1.234, requires_grad=True)
print("x device:", x.device)

x = x.to(device)
print("x device after move:", x.device)

y = torch.sin(x) * x**2
y.backward()

print("dy/dx:", x.grad)

## Models and devices

Models are just collections of tensors.

Moving a model to a GPU moves *all of its parameters*.

In [ ]:
# Let's move a model to the GPU if available and perform a forward pass

toymodel = ToyModel() # initialize the model object
toymodel = toymodel.to(device) # move the model to the selected device

# print the model parameters and their devices
for name, p in toymodel.named_parameters():
    print(f"parameter name: {name}, value: {p.item()}, requires_grad: {p.requires_grad}, device: {p.device} ")


Everything else behaves the same way as on CPU.

In [ ]:
# perform a forward pass on the selected device
target = torch.tensor(1.0).to(device) # move target to the selected device
toyoptimizer = torch.optim.SGD(toymodel.parameters(), lr=1E-3) # create an SGD optimizer to update model parameters, nothing different here 

# Doing one optimization step
toyoptimizer.zero_grad()
y = toymodel()
loss = (y - target)**2
loss.backward()
toyoptimizer.step()

print("x =", toymodel.x.item(), "loss =", loss.item())

# <u> Minimal MLP demo: fitting a parabola </u>

We’ll train a tiny neural network to learn a function from data.

**Task:** learn \( y = x^2 \) from sampled points.

This will highlight:
- building a model (`nn.Module`)
- training loop with `torch.optim`
- a minimal `Dataset` + `DataLoader`
- (optional) GPU offloading via `.to(device)`

In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

## Synthetic dataset

We sample "x" uniformly and generate:

y = x^2 + ε

where "ε" is small Gaussian noise.

In [ ]:
# define a random number seed for reproducibility
torch.manual_seed(0)

# sample synthetic dataset from the function y = x^2 + noise
N = 2048 # number of data points
x = 2.0 * torch.rand(N, 1) - 1.0          # x in [-1, 1], shape (N, 1)
y = x**2 + 0.03 * torch.randn(N, 1)        # noisy parabola, shape (N, 1)

# Train/val split
n_train = int(0.8 * N) # number of training data points, 80% for training and 20% for validation
x_train, x_valid = x[:n_train], x[n_train:]
y_train, y_valid = y[:n_train], y[n_train:]

train_ds = TensorDataset(x_train, y_train) # create training dataset
val_ds   = TensorDataset(x_valid, y_valid) # create validation dataset
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True) # create training data loader
valid_loader   = DataLoader(val_ds, batch_size=256, shuffle=False) # create validation data loader

print("train batches:", len(train_loader), "val batches:", len(valid_loader))

## Model: a tiny MLP

We’ll use a fully-connected network:

- input: 1 number (x)
- hidden: 32 → 32
- output: 1 number (y)

Activation: **ReLU** 

In [ ]:
# Define the model: a tiny MLP
class ParabolaMLP(nn.Module):
    def __init__(self, hidden=32):
        super().__init__()
        self.hidden = hidden # store hidden size
        
        self.net = nn.Sequential(
            nn.Linear(1, self.hidden), # input layer size: (x) 1 -> hidden size
            nn.ReLU(), # activation function
            nn.Linear(self.hidden, self.hidden), # hidden layer: hidden size -> hidden size
            nn.ReLU(), # activation function
            nn.Linear(self.hidden, 1), # output layer size: hidden size -> 1 (y)
        )

    def forward(self, x):
        return self.net(x)

mlp = ParabolaMLP(hidden=32).to(device)
print(mlp)

## Training setup

Loss function: Minimize mean squared error (MSE)

Optimizer: Adam

In [ ]:
loss_fn = nn.MSELoss() # loss function
mlpopt = optim.Adam(mlp.parameters(), lr=1E-3) # optimizer

## Training loop

Each epoch:
1. loop over mini-batches from the DataLoader
2. forward pass → loss
3. `loss.backward()` computes gradients
4. `opt.step()` updates parameters

In [ ]:
# make code modular by defining an evaluation function
def evaluate(model, loader):
    
    model.eval() # change model to evaluation mode, disabling dropout/batchnorm if any
    total_loss = 0.0 # initialize total loss
    n = 0 # initialize number of samples
    
    with torch.no_grad():
        # iterate over batches in the data loader
        for xb, yb in loader:

            xb = xb.to(device) # move input batch to device
            yb = yb.to(device) # move target batch to device

            pred = model(xb) # make model prediction
            loss = loss_fn(pred, yb) # compute loss
            
            total_loss += loss.item() * xb.size(0) # accumulate total loss over all samples
            n += xb.size(0) # update number of samples
            
    return total_loss / n # return average loss over all samples

# define number of training epochs
epochs = 100

# training loop
for epoch in range(1, epochs + 1):
    mlp.train() # change model to training mode, turns on dropout/batchnorm if any
    for xb, yb in train_loader:
        xb = xb.to(device) # move input batch to device
        yb = yb.to(device) # move target batch to device

        ypred = mlp(xb) # forward pass
        loss = loss_fn(ypred, yb) # compute loss

        loss.backward() # backward pass to compute gradients

        mlpopt.step() # update model parameters in the optimizer

        mlpopt.zero_grad() # clear existing gradients

    # evaluate and print losses every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        train_loss = evaluate(mlp, train_loader) # evaluate on training set
        valid_loss = evaluate(mlp, valid_loader) # evaluate on validation set
        print(f"epoch {epoch:03d} | train log(MSE)={np.log10(train_loss):.5f} | val log(MSE)={np.log10(valid_loss):.5f}")

## Visualizing the learned function

We compare:
- the noisy training data
- the true function \(y = x^2\)
- the neural network prediction

This helps us see what the MLP actually learned.

In [ ]:
import matplotlib.pyplot as plt # import matplotlib for plotting

# Put model in eval mode
mlp.eval()

# Generate a smooth x grid
x_plot = torch.linspace(-1.0, 1.0, 400).unsqueeze(1).to(device)

with torch.no_grad():
    y_pred = mlp(x_plot).cpu()

# Move to CPU for plotting
x_plot = x_plot.cpu()
y_true = x_plot**2

fig,ax = plt.subplots(figsize=(6, 4),layout='constrained')

# Training data (subsample for clarity)
ax.scatter(
    x_train[::10].numpy(),
    y_train[::10].numpy(),
    s=10,
    alpha=0.4,
    label="training data",
)

# True function
ax.plot(
    x_plot.numpy(),
    y_true.numpy(),
    linewidth=2,
    label="true y = x²",
)

# Model prediction
ax.plot(
    x_plot.numpy(),
    y_pred.numpy(),
    linewidth=2,
    linestyle="--",
    label="MLP prediction",
)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()
plt.show()